In [ ]:
import os 
import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt

import jou_gen as jg

from io import StringIO

import scipy.optimize as sco
from sklearn.metrics import mean_squared_error

import plotly.graph_objects as go

import hx_hydraulic as hxhy

In [ ]:
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.41\1 channel\fluent\parametric-study-flow-rate'

In [ ]:
inputs = pd.read_excel(folder_path + r'\inputs.xlsx', engine='openpyxl')

In [ ]:
inputs

In [ ]:
40/16/1000/3600

In [ ]:
channel = hxhy.duct("rectangular",h=0.002,w=0.003,L=1.6)
channel.set_fluid()

In [ ]:
for row, value in inputs.iterrows():
    channel.set_mdot(inputs.loc[row, 'mdot_ch'])
    channel.compute_Reynolds()
    inputs.loc[row, 'Re'] = channel.Re

In [ ]:
C = 1323000

In [ ]:
journal_list = []

In [ ]:
# solve/initialize/hyb-initialization yes

In [ ]:
for row, value in inputs.iterrows():
    journal_list.append(jg.change_named_expression('mdot_ch', value['mdot_ch'], 'kg/s'))

    if value['Re'] > 1000:
        journal_list.append("""define/models/viscous/kw-sst yes""")

    if row == 0:
        journal_list.append(jg.standard_initialization(1, C*value['mdot_ch']))

    journal_list.append(jg.iterate(value['nb_it']))
    journal_list.append(jg.write_residuals_file(folder_path+f"\\{value['test']}_residuals.txt"))
    journal_list.append(jg.write_report(surface_list = np.arange(0,26,1), file_path=folder_path+f"\\{value['test']}_sp_report.txt"))
    journal_list.append(jg.write_time(file_path=folder_path+f"\\{value['test']}_time.txt"))
    journal_list.append(jg.write_data(file_path=folder_path+f"\\{value['test']}_data.txt"))                    

In [ ]:
jg.concatenate_and_write_to_file(journal_list, folder_path + r'\parametric-journal.txt')

In [ ]:
inputs.loc[6]['mdot_ch']*C

In [ ]:
inputs

In [ ]:
pressure_report = {}

for i, row in inputs.iterrows():
    with open(folder_path+f'\\{inputs.loc[i, "test"]}_sp_report.txt', 'r') as file:
        text_content = file.read() 
    pressure_report[inputs.loc[i, 'test']] = jg.parse_report_to_dataframe(text_content, 'Surface Integral Report')

In [ ]:
for i, row in inputs.iterrows():
    case = inputs.loc[i, 'test']

    inputs.loc[i, 'DP'] = pressure_report[case][pressure_report[case]['Component'] == 'internal_ab']['Surface Integral Report'].values[0] - pressure_report[case][pressure_report[case]['Component'] == 'internal_fg']['Surface Integral Report'].values[0]

In [ ]:
popt, pcov = sco.curve_fit(lambda x,a : a*x, inputs.loc[inputs['mdot_ch']<0.002]['mdot_ch'], inputs.loc[inputs['mdot_ch']<0.002]['DP'])

In [ ]:
inputs

In [ ]:
# Scatter plot for the data points
scatter = go.Scatter(x=inputs['mdot_ch'], y=inputs['DP'], mode='markers', marker=dict(symbol='cross'), name='Data Points')

# Line plot for the fitted line
fitted_line = go.Scatter(x=inputs['mdot_ch'], y=popt[0]*np.array(inputs['mdot_ch']), mode='lines', name='Fit')

# Creating the figure
fig = go.Figure(data=[scatter, fitted_line])

# Update layout for the figure
fig.update_layout(
    title='Pressure Drop vs. Mass Flow Rate',
    xaxis_title='Mass flow rate [kg/s]',
    yaxis_title='Pressure drop [Pa]'
)

# Show plot
fig.show()


In [ ]:
# Compute the predicted values using the fit equation
predicted_values = popt[0] * inputs.loc[inputs['mdot_ch']<0.002]['mdot_ch']

# Compute the RMSE
rmse = mean_squared_error(inputs.loc[inputs['mdot_ch']<0.002]['DP'], predicted_values, squared=False)
rmse/np.mean(inputs.loc[inputs['mdot_ch']<0.002]['DP'])

In [ ]:
0.002*3600*50

In [ ]:
25E-6*3600*300

In [ ]:
(1411785-1323000)/1411785

In [ ]:
popt